## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test, mean_train, std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot = OneHotEncoder()
y_train = one_hot.fit_transform(y_train).toarray()
y_test = one_hot.transform(y_test).toarray()

/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:
classifier = Sequential()

#卷積組合 
classifier.add(Convolution2D(64, kernel_size=(3, 3), padding='same', input_shape=(32, 32, 3)))
classifier.add(BatchNormalization())
classifier.add(Activation('sigmoid'))

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3, 3), padding='same'))
classifier.add(BatchNormalization())

classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(16, kernel_size=(5, 5), padding='same'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim = 100, activation='relu'))
               
#輸出
classifier.add(Dense(output_dim = 10, activation='softmax'))

classifier.summary()













_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        18464     
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 16)        12816    

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100)`
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


In [5]:
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train, y_train, batch_size=100, epochs=100)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
50000/50000 [==============================] - 202s 4ms/step - loss: 1.3867 - acc: 0.5109
Epoch 2/100
50000/50000 [==============================] - 213s 4ms/step - loss: 0.9711 - acc: 0.6598
Epoch 3/100
50000/50000 [==============================] - 201s 4ms/step - loss: 0.8174 - acc: 0.7155
Epoch 4/100
50000/50000 [==============================] - 202s 4ms/step - loss: 0.7269 - acc: 0.7454
Epoch 5/100
50000/50000 [==============================] - 201s 4ms/step - loss: 0.6436 - acc: 0.7731
Epoch 6/100
50000/50000 [==============================] - 196s 4ms/step - loss: 0.5764 - acc: 0.7974
Epoch 7/100
50000/50000 [==============================] - 197s 4ms/step - loss: 0.5145 - acc: 0.8208
Epoch 8/100
50000/50000 [==============================] - 196s 4ms/step - loss: 0.4551 - acc: 0.8404
Epoch 9/100
50000/50000 [==============================] - 200s 4ms/step - loss: 0.399

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [6]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[8.3718694e-09, 9.9227411e-25, 8.0547119e-07, 9.9999630e-01,
        2.0868081e-06, 1.2787728e-12, 1.6190929e-13, 3.1187218e-14,
        7.8865298e-07, 3.2949112e-15]], dtype=float32)